## 1 Import some lib for further use

In [ ]:
# general imports
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## 2 Introduce the target model

In [ ]:
target_model=tf.keras.models.load_model("/content/trained_target_model.h5")

## 2a See the structure of target model, takes image, output vector (10,)

In [ ]:
target_model.summary()

1st layer: Conv layer + Max Pooling

2nd layer: Conv layer + Max Pooling

3rd layer: Fully Conn (256 * 128)

4th layer: Fully COon (128 * 10)

## 3 Load some data from .npy files

In [ ]:
p_train_data=np.load("/content/train_data_partial.npy")

## 3a What is data look like?

In [ ]:
#p_train_data.shape

In [ ]:
# show some train data: first 10 data
def show_data(image_data):
        num_row = 2
        num_col = 5
        fig, axes = plt.subplots(num_row, num_col, figsize=(1.5*num_col,2*num_row))
        for i in range(10):
            ax = axes[i//num_col, i%num_col]
            ax.set_axis_off()
            ax.imshow(image_data[i])

        plt.tight_layout()
        plt.show()

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

## 3b Print some data to SEE

In [ ]:
show_data(p_train_data)

## 3c Labels of the data

In [ ]:
p_train_labels=np.load("/content/train_labels_partial.npy")

In [ ]:
p_train_labels.shape

In [ ]:
p_train_labels

## 3d Data labels by names

In [ ]:
first_10_labels=p_train_labels[:10]
for idx in first_10_labels:
    print(class_names[idx])

## 4 How the model work? Model(data)=output

### 4a Prepare random data for test

In [ ]:
rand_idx=np.random.randint(0,10000, (100,))

In [ ]:
rand_data=p_train_data[rand_idx]

In [ ]:
show_data(rand_data)

### 4b Get labels as groundtruth of random data

In [ ]:
first_10_labels=p_train_labels[rand_idx]
print(first_10_labels)
for idx in first_10_labels:
    print(class_names[idx])

## 4c Use target model to predict random data?

In [ ]:
prediction=target_model(rand_data)

In [ ]:
prediction.shape

In [ ]:
prediction[1]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([ -4.2143583,  -9.740183 ,   1.2927963,  -1.5680918,   0.7099859,
        -4.244934 ,  10.501431 ,  -8.168477 ,  -8.988067 , -11.414565 ],
      dtype=float32)>

### 4d What is the predicted label?

In [ ]:
predict_label=tf.math.argmax(prediction, axis=1)
print(predict_label)

In [ ]:
for idx in predict_label:
    print(class_names[idx])

### 4e Accuracy of prediction on the 10 test samples

In [ ]:
pred_accuracy=sum(first_10_labels==predict_label.numpy())/len(first_10_labels)
print(pred_accuracy)

0.93


=============================================================

## Attack preparation

### 1 Load training data, the members (We have done)
### 2 Load testing data, the non-members
### 3 Input member and non-members into target model
### 4 Get statistics for training your attack model

### 1 

In [ ]:
# p_train_data=np.load("/content/train_data_partial.npy")
# p_train_data is the training data, we call "Members"

### 2

In [ ]:
p_test_data=np.load("/content/test_data_partial.npy")
p_test_data

array([[[[0.61960787, 0.4392157 , 0.19215687],
         [0.62352943, 0.43529412, 0.18431373],
         [0.64705884, 0.45490196, 0.2       ],
         ...,
         [0.5372549 , 0.37254903, 0.14117648],
         [0.49411765, 0.35686275, 0.14117648],
         [0.45490196, 0.33333334, 0.12941177]],

        [[0.59607846, 0.4392157 , 0.2       ],
         [0.5921569 , 0.43137255, 0.15686275],
         [0.62352943, 0.44705883, 0.1764706 ],
         ...,
         [0.53333336, 0.37254903, 0.12156863],
         [0.49019608, 0.35686275, 0.1254902 ],
         [0.46666667, 0.34509805, 0.13333334]],

        [[0.5921569 , 0.43137255, 0.18431373],
         [0.5921569 , 0.42745098, 0.12941177],
         [0.61960787, 0.43529412, 0.14117648],
         ...,
         [0.54509807, 0.38431373, 0.13333334],
         [0.50980395, 0.37254903, 0.13333334],
         [0.47058824, 0.34901962, 0.12941177]],

        ...,

        [[0.26666668, 0.4862745 , 0.69411767],
         [0.16470589, 0.39215687, 0.5803922 ]

In [ ]:
p_test_data.shape

(5000, 32, 32, 3)

### 3

In [ ]:
member_stat=target_model(p_train_data)

In [ ]:
# This information is the "attrubutes" of member data
member_stat

<tf.Tensor: shape=(10000, 10), dtype=float32, numpy=
array([[ -8.09453   ,  -9.511581  ,   2.308946  , ...,   0.7989033 ,
        -11.765329  , -14.546907  ],
       [  1.4993261 ,  -1.5915823 ,   2.2405732 , ...,  -3.7230422 ,
         -5.4061685 ,   9.783438  ],
       [  1.2942592 ,   1.0332361 ,  -0.7563736 , ...,  -5.567007  ,
         -7.7828164 ,   8.665651  ],
       ...,
       [  0.69104695,   7.2425804 ,  -4.5836205 , ...,  -4.4953585 ,
          2.8515415 ,   7.202874  ],
       [ -1.0489266 ,  10.469152  ,  -2.3504214 , ...,  -3.8080924 ,
         -3.2119243 ,   3.6190464 ],
       [ -4.1086407 ,  -7.4670677 ,   3.2689018 , ...,   6.2206845 ,
        -11.102761  ,  -9.658786  ]], dtype=float32)>

In [ ]:
non_member_stat=target_model(p_test_data)

In [ ]:
# This information is the "attrubutes" of member data
non_member_stat

<tf.Tensor: shape=(5000, 10), dtype=float32, numpy=
array([[ -1.7871485 ,  -5.1116824 ,  -1.3135241 , ...,  -9.017349  ,
          3.2459524 ,  -4.4019    ],
       [  6.0741    ,   3.6214423 ,  -5.7152243 , ...,  -8.691189  ,
         11.031433  ,   1.1811364 ],
       [  0.8379353 ,   3.4393337 ,  -5.3589616 , ...,  -2.32182   ,
          3.0722766 ,   1.130228  ],
       ...,
       [ -3.689346  ,  -5.9777665 ,  -3.7441618 , ...,  -1.4427079 ,
         -5.288816  ,  -2.697714  ],
       [-14.829874  ,  -8.71036   ,   3.8072836 , ...,   0.63230956,
        -10.641227  , -12.714217  ],
       [  1.2543784 ,  15.432904  ,  -4.1131434 , ..., -12.167717  ,
         -8.198457  ,   5.549427  ]], dtype=float32)>

## 4 Knowledge we have now
### A binary dataset with the form:

### {Member_stat:1, Non-member_stat:0}

## 5 Attack process: use the knowledge to train a BINARY classisier MIA

### MIA(x)={0, 1}

### MIA(X)=[0,0,0,0,1,1,1,1........0,1,0,1,0,1]

In [ ]:
# Your Attack code produce your MIA attack model
# Attack code
def normalize(lst):
  for item in lst:
    minval = min(item)
    maxval = max(item)
    for i in range(len(item)):
      item[i] = (item[i] - minval)/(maxval-minval)
  return lst

def calculate_threshold(lst):
  sum = 0
  totalsum = 0
  diffsum = 0
  for item in lst:
    maxval = max(item)
    sum = 0
    for i in range(len(item)):
      sum += maxval - item[i]
    totalsum += sum/len(item)

  threshold = totalsum/len(lst)
  return threshold

members_data = member_stat.numpy()

members_data = normalize(members_data)

non_members_data = non_member_stat.numpy()

non_members_data = normalize(non_members_data)

members_threshold = calculate_threshold(members_data)

non_members_threshold = calculate_threshold(non_members_data)
print(members_threshold)

print(non_members_threshold)

0.5471832224637251
0.5423435038530824


In [ ]:
def MIA(data):
  predictions = target_model(data)
  predictions = predictions.numpy()
  predictions = normalize(predictions)
  sum = 0
  ans = []
  for tp in predictions:
    maxval = max(tp)
    sum = 0
    for i in range(len(tp)):
      sum += maxval - tp[i]
    sum /= len(tp)
    if(sum >= non_members_threshold):
      ans.append(1)
    else:
      ans.append(0)
  return ans

## 6 Evaluate your MIA attack model

### We provide "test_member_data.npy" and "test_non-member_data.npy"

Evaluating test member data

In [44]:
test_member_data = np.load("./test_member_data.npy")
test_member_result = MIA(test_member_data)

ones = 0
for x in test_member_result:
  if(x == 1):
    ones += 1
accuracy = (ones/len(test_member_result))*100
print(accuracy)

50.6


Evaluating non member data

In [43]:
test_non_member_data = np.load("./test_Nonmember_data.npy")
test_non_member_result = MIA(test_non_member_data)

zeros = 0
for x in test_non_member_result:
  if(x == 0):
    zeros += 1
accuracy = (zeros/len(test_non_member_result))*100
print(accuracy)

53.0


## 7 How do we grade your model?

In [ ]:
# You need to load the evaluation data
eval_data=np.load("./evaluation_data.npy")

In [ ]:
# Infer the membership of evaluation data by your MIA model
Your_evaluation_result=MIA(eval_data)

Saving result to .npy file

In [46]:
np.save("BhargavMuppalla_evaluation_result.npy",Your_evaluation_result)

### 7a The evaluation result should be the form of [0,0,0,0,1,1,1,1........0,1,0,1,0,1]
### 7b The length of result is equal to length of evaluation dataset

### 7c We compare your evaluation result and our ground truth membership

### Calculate your MIA accuracy

In [ ]:
MIA_accuracy=sum(Your_evaluation_result==Ground_truth_membership)/len(eval_data)

NameError: ignored